In [ ]:
import os
import re
import tempfile
import warnings
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from google3.learning.deepmind.xmanager2.client import xmanager_api
from google3.pyglib import gfile
from google3.pyglib.function_utils import memoize
from matplotlib import font_manager

import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.lines import Line2D
from matplotlib.ticker import FixedLocator  # Import for the fix
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import LogLocator
from matplotlib.ticker import FuncFormatter

In [ ]:
#@title Google Sans Import

# Import Google font family
_GOOGLE_SANS_PATH = (
    'google3/third_party/googlefonts/api/googlerestricted/googlesans/'
)

@memoize.Memoize()
def import_google3_fonts(font_path: str) -> None:
  """Import fonts stored in google3 into Matplotlib for use in Colab.

  Args:
    font_path: google3 path to either a directory that contains .ttf fonts or to
      a specific .ttf font file.
  """
  if gfile.IsDirectory(font_path):
    # Create a temp directory as a destination for copied font files.
    tmp_dir = tempfile.mkdtemp()
    # Copy font files from google3 to temp dir.
    gfile.RecursivelyCopyDir(font_path, tmp_dir, overwrite=True)
    # Add font files in directory to matplotlib font_manager.
    font_files = font_manager.findSystemFonts(fontpaths=tmp_dir)
  else:
    # Assume the path points to a file if it's not a directory.
    # Copy ttf file from google3 to temp location.
    tmp_file = tempfile.NamedTemporaryFile(suffix='.ttf')
    tmp_file.close()
    gfile.Copy(font_path, tmp_file.name)
    font_files = [tmp_file.name]

  # Add fonts to default font manager.
  for font_file in font_files:
    font_manager.fontManager.addfont(font_file)


def import_default_google_fonts() -> None:
  """Register a set of default fonts (Roboto, Google Sans) with Matplotlib."""
  # Prepend google_src to google3 paths.
  import_google3_fonts(os.path.join('/google_src/head/depot', _GOOGLE_SANS_PATH))


# Import and register Google fonts with Matplotlib so we can use them.
import_default_google_fonts()

In [ ]:
#@title Set up Plot Settings

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
# Suppress specific warning
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

xm_client = xmanager_api.XManagerApi(xm_deployment_env='alphabet')
MEDIUM_SIZE = 12
mpl.rcParams.update({
    'font.size': MEDIUM_SIZE,
    'axes.labelsize': MEDIUM_SIZE,
    'axes.titlesize': MEDIUM_SIZE,
})
plt.rcParams['font.size'] = MEDIUM_SIZE
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['axes.edgecolor'] = '#777777'
plt.rcParams['axes.facecolor'] = '#FFFFFF'
plt.rc('font', size=MEDIUM_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE-5)  # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

elegant_palette = sns.color_palette('muted')
mpl.rcParams['font.family'] = 'Google Sans'

In [ ]:
# Create the figure and axes for the subplots
def log_float_formatter(y, pos):
    return f'{y:.2f}'

#@title Data vs. Subject Scaling

map_results = {
    'num_of_shots': [1000*5, 10000*5, 100000*5, 750000*5, 1321235*5],
    'lsm_ft_ed': [68.34999999996583, 73.84999999996307, 84.94999999995753, 87.44999999995628, 90.29999999995485],
    'lsm_ft_ar': [51.50214592273099, 64.00980993253708, 68.79215205393353, 64.1630901287357, 68.48559166153633],
    'lsm_lp_ar': [35.13182096872007, 39.607602697719315, 47.057020232971475, 47.60882893928645, 49.448191293669694],
    'lsm_lp_ed': [60.5999999999697, 67.34999999996633, 84.54999999995772, 88.04999999995597, 84.69999999995765]
    # 'supervised_baseline': [12.781144280731132, 13.56088290669249, 13.222288420289424, 15.153588158587056]
}


# df = pd.DataFrame(map_results)

fig, axes = plt.subplots(1, 1, figsize=(3.4, 3), sharex=True, dpi=100)
axes.set(xscale="log", yscale="log")
# Create a color palette using the "Blues" color map
palette = sns.color_palette("Blues", n_colors=3)

# Plot the lines with the "Blues" palette
sns.lineplot(
    data=map_results,
    x='num_of_shots',
    y='lsm_ft_ar',
    color=palette[2],
    marker='o',
    label='Activity Recognition',
    markersize=5,
    linestyle='--',
)

sns.lineplot(
    data=map_results,
    x='num_of_shots',
    y='lsm_ft_ed',
    color=palette[1],
    marker='o',
    label='Exercise Detection',
    markersize=5,
    linestyle='--',
)
axes.yaxis.set_major_locator(LogLocator(base=10.0, subs=np.arange(1, 10), numticks=10))
axes.yaxis.set_major_formatter(FuncFormatter(log_float_formatter))
axes.xaxis.set_major_locator(LogLocator(base=10.0, subs=np.arange(1, 10), numticks=10))
axes.set_ylabel('Accuracy (%)')
axes.set_xlabel('Pretraining Data Size (Hours)')
plt.legend(frameon=False, fontsize=MEDIUM_SIZE-2)
plt.tight_layout()
plt.savefig("/tmp/scaling_dis_acc.pdf", bbox_inches='tight', format="pdf")
%download_file /tmp/scaling_dis_acc.pdf
plt.show()
